Here, I have taken a sample section of the csv file that we have our data on and added in any missing blocks, for now I have dropped gas columns but will try to add back in via alchemy api as demonstrated in the first few lines of code. It's a lot of data to add back in so will be simple, but painful. For now, we can run the models without wgv.

In [32]:
import numpy as np
import pandas as pd
import requests
from web3 import Web3, HTTPProvider

In [28]:
url = "https://mainnet.gateway.pokt.network/v1/lb/62fd2e8c466be4003a7ac327/v1/query/blocktxs" # my free alchemy historical api 

In [29]:
r = requests.post(url) # post request 

In [44]:
# Initialize a Web3.py instance
web3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/PXDcncN7KHOchKhO4S9FzldLoXBtshEI'))

# Query the blockchain (replace example parameters)
hist = web3.eth.fee_history(newest_block = 11834049, block_count=1000) # just picked first block from df below

# Print the output to console
print(hist)   # will be using this to get gas levels for all non-MEV blocks

AttributeDict({'oldestBlock': 11833050, 'baseFeePerGas': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [76]:
len(hist.gasUsedRatio) # this api returns aount of requests, will look into batching to get a lot more

1000

In [3]:
df = pd.read_csv('mev_sample.csv')  # use this to load in prior CSV

In [51]:
df.drop('MEV_category2',axis=1,inplace=True)  # drop any columns needed, in this instance drop prior 'MEV_category' column as we are re standardising based on blocks, not transactions

In [48]:
df['miner_reward'].describe()  # shows interquartile ranges, we use this in condition below to categorise MEV into sections

count    5.000000e+03
mean     1.902100e+17
std      7.944668e+17
min      7.561440e+13
25%      3.660000e+16
50%      8.865000e+16
75%      1.810000e+17
max      4.020000e+19
Name: miner_reward, dtype: float64

In [49]:
def condition(x):
    if 0 <= x <= list(df['miner_reward'].describe())[4]:
         return 1
    elif list(df['miner_reward'].describe())[4] < x <= list(df['miner_reward'].describe())[5]:
        return 2
    elif x > list(df['miner_reward'].describe())[6]:
        return 3

In [53]:
df['MEV_category'] = df['miner_reward'].apply(condition)

In [56]:
df2 = (df.set_index('block_number')
   .reindex(range(df.block_number.iat[0],df.block_number.iat[-1]+1), fill_value=0)
   .reset_index())   # get all blocks filled in (including 0 MEV blocks)

In [62]:
df2.head()

,block_number,miner_reward,gas_used,gas_price,MEV_category
0,11834049,3.720000e+17,4505938,8.261322e+10,3.0
1,11834050,0.000000e+00,0,0.000000e+00,0.0
2,11834051,0.000000e+00,0,0.000000e+00,0.0
3,11834052,0.000000e+00,0,0.000000e+00,0.0
4,11834053,0.000000e+00,0,0.000000e+00,0.0


In [65]:
df2.drop('gas_used',axis=1,inplace=True) # drop columns for now, we can use gas later if needed

In [66]:
df2.drop('gas_price',axis=1,inplace=True) # use 'inplace = True' to make adjustment to df itself - don't use this arguent if you just want to look at df withough the dropped column

In [67]:
df2.head()

,block_number,miner_reward,MEV_category
0,11834049,3.720000e+17,3.0
1,11834050,0.000000e+00,0.0
2,11834051,0.000000e+00,0.0
3,11834052,0.000000e+00,0.0
4,11834053,0.000000e+00,0.0


In [69]:
df2['MEV_category'].value_counts()

0.0    285348
1.0      1251
2.0      1249
3.0      1244
Name: MEV_category, dtype: int64